# EMGdecomPy Workflow

This notebook allows for simple visualisation of the decomposition results obtained from the `EMGDecomPy` package.

## Imports - dependencies

Please run the cells below to add all the dependencies to run this notebook.   
First, import the Python packages for loading in and saving data:

In [1]:
# loadmat is used to load MATLAB data
from scipy.io import loadmat

# Pickle allows saving and loading Python objects into a file
import pickle

## Imports - EMGDecomPy

Now, import the `EMGDecomPy` package scripts for decomposing and visualizing the electromyography data:

In [2]:
from emgdecompy.decomposition import *
from emgdecompy.contrast import *
from emgdecompy.viz import *
from emgdecompy.preprocessing import *

## Load the raw EMG data

Please specify the filepath where the raw data is located, and use it to define the  `raw_data` variable.

This raw data will be processed via an algorithm based on the the process proposed by `Negro et al. (2016)`.

In order for the decomposition to accurately process the data, it needs to be in the correct format. The `decomposition` function and the `visualize_decomp` functions expect the `raw` argument to be a numpy array of arrays, where each inner element array is a time series of electromyography signal observations per channel.

For instance, during this project, we used data in the MATLAB `.mat` format. A simplified example of the raw data dictionary can be seen below.

Below, we load the data via the `loadmat` function, which we index into its signal series with the `SIG` key.

In [3]:
raw = loadmat('../data/raw/gl_10.mat')['SIG']

Note the aforementioned `SIG` key-value pair, containing the array of arrays. For the purposes of decomposition, `SIG` is the only key needed within the dictionary. However, the rest of the information stored within the `.mat` file can be found below:

## Run decomposition

Run the decomposition on raw data to retrieve a dictionary containing the results of the blind source separation algorithm. Further documentation for each function can be found [here](https://emgdecompy.readthedocs.io/en/stable/autoapi/emgdecompy/index.html).

__There is a number of parameters you may choose to customize:__

| Parameter    | Data Type                    | Explanation                                                                                  |
|--------------|------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| x            | numpy.ndarray                | Raw EMG signal. This is expected inside the 'SIG' key within the `.mat` file loaded above.  
| discard      | slice, int, or array of ints | Indices of channels to discard. Useful if you are looking to only analyze some channels, or expect some channels to contain significant noise.                               |
| R            | int                          | How far to extend x during the extension step of the algorithm.                                                                                                            |
| M            | int                          | Number of iterations to run the decomposition for.                                                                                                                             |
| bandpass       | bool                         | Whether to band-pass filter the raw EMG signal or not.                                                                                                                     |
| lowcut       | float                        | Lower range of band-pass filter.                                                                                                                                           |
| highcut      | float                        | Upper range of band-pass filter.                                                                                                                                           |
| fs           | float                        | Sampling frequency in Hz.                                                                                                                                                  |
| order        | int                          | Order of band-pass filter.                                                                                                                                                 |                                                                                                                          |
| Tolx         | float                        | Tolerance for element-wise comparison in separation.                                                                                                                       |
| contrast_fun | function                     | Contrast function to use. `skew`, `og_cosh` or `exp_sq`                                                                                                                          |
| ortho_fun    | function                     | Orthogonalization function to use. `gram_schmidt` or `deflate`                                                                                                                 |
| max_iter_sep | int > 0                      | Maximum iterations for fixed point algorithm.                                                                                                                              |
| l            | int                          | Required minimal horizontal distance between peaks in peak-finding algorithm. Default value of 31 samples is approximately equivalent to 15 ms at a 2048 Hz sampling rate. |
| sil_pnr      | bool                         | Whether to use SIL or PNR as the acceptance criterion. Default value of True uses SIL.                                                                                         |
| thresh       | float                        | SIL/PNR threshold for accepting a separation vector.                                                                                                                       |
| max_iter_ref | int > 0                      | Maximum iterations for refinement.                                                                                                                                         |
| random_seed  | int                          | Used to initialize the pseudo-random processes in the function.                                                                                                            |
| verbose      | bool                         | If true, decomposition information is printed.                                                                                                                             |


In [4]:
# output = decomposition(
#     raw,
#     discard=5,
#     R=16,
#     M=64,
#     bandpass=True,
#     lowcut=10,
#     highcut=900,
#     fs=2048,
#     order=6,
#     Tolx=10e-4,
#     contrast_fun=skew,
#     ortho_fun=gram_schmidt,
#     max_iter_sep=10,
#     l=31,
#     sil_pnr=True,
#     thresh=0.9,
#     max_iter_ref=10,
#     random_seed=None,
#     verbose=False
# )

The output of decomposition is a dictionary containing the following keys:

__'B'__: Matrix whose columns contain the accepted separation vectors.

__'MUPulses'__: Firing indices for each motor unit. This is the key used within this notebook, along with the raw data, for visualisation purposes.

__'SIL'__: Corresponding silhouette scores for each accepted source.

__PNR__: Corresponding pulse-to-noise ratio for each accepted source.

A simplified example of the output of the `decomposition` function can be seen below:

## Optional: save the output object

Cell below saves the output of the decomposition.

In [5]:
# decomp_sample = output 
# decomp_sample_pkl = open('decomp_sample_pkl.obj', 'wb') 
# pickle.dump(decomp_sample, decomp_sample_pkl)

Cell below loads the output saved above.

In [6]:
with open('decomp_sample_pkl.obj', 'rb') as f: output = pickle.load(f)

## Visualize results

Finally, run the cells below to get an interactive dashboard of the decomposed data.

There are two parameters for the `visualize_decomp` function. `decomp_results` and `raw` take in the decomposition results and raw signal, respectively. You are unlikely to want to change it within the function, and most likely will select those earlier. However, if you want to plug in the data directly, you may use those parameters.

There are three widgets controlling the rest of the dashboard:   
1. The `Motor Unit` widget for selecting the Motor Unit to examine.
2. The `Preset` parameter allows you to select arrangements of the channel. Currently, you may select between 'standard' and 'vert63' arrangements. New arrangements may easily be added within the `channel_preset` function in the `viz.py` script.
3. The `Method` widget allows you to select the metric for calculating the average mismatch score between peak contribution and average motor unit action potential (MUAP) shape. Currently, RMSE is the only available metric, but other metrics can be added easily in the future by adding functions to the `viz.py` script.

Below the widgets, dashboard contains four charts:

#### Chart 1: Signal Strength + Firing Rate. 
The top chart displays signal strength along with the instanteneous firing rate at that point.   
This chart allows for interval control. Select the interval of interest by clicking and dragging the mouse. Move around the interval by clicking on the selection and moving it around. The plots below zoom in according to this selection.

#### Chart 2: Firing Rate. 
The second chart displays the instanteneous firing rate of each spike. Outliers may be indicative of a false positive firing. Click on an individual data point to select corresponding the pulse. This will allow you to see the given firing's contribution to the MUAP shape.

#### Chart 3: Signal Strength
The third chart displays the signal strength of each firing. Click on an individual data point to select corresponding pulse. This will allow you to see the given firing's contribution to the MUAP shape.

#### Chart 4: MUAP Shapes + Peak Contributions
The last chart displays the average shape for each channel. The title displays the currently selected motor unit. If an individual peak has been selected using Chart 2 or 3, then an overlay will be displayed on each sub-chart that shows the contribution of the selected peak towards the shape for each channel. You may toggle the opacity of that overlay and the MUAP shape using the legend. Additionally, the peak firing time and RMSE will displayed in the title of the chart.

Further documentation of the visualization functions can be found [here](https://emgdecompy.readthedocs.io/en/stable/autoapi/emgdecompy/viz/index.html).

In [7]:
visualize_decomp(output, raw)

Column
    [0] Column
        [0] Select(name='Motor Unit:', options=[0, 1, 2, 3, 4, ...], value=0)
        [1] Select(name='Preset:', options=['standard', 'vert63'], value='standard')
        [2] Select(name='Comparison Metric:', options=['RMSE'], value='RMSE')
    [1] Row
        [0] Column
            [0] Vega(VConcatChart, debounce=10, selection=Selection)
            [1] ParamFunction(function)